# generate google moderation labels for test set

to authenticate, run

```
gcloud auth application-default login
```

In [15]:
def detect_safe_search_local(path):
    """Detects unsafe features in the file."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient(credentials=CREDS)

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.safe_search_detection(image=image)
    safe = response.safe_search_annotation

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = (
        "UNKNOWN",
        "VERY_UNLIKELY",
        "UNLIKELY",
        "POSSIBLE",
        "LIKELY",
        "VERY_LIKELY",
    )
    print("Safe search:")

    print(f"adult: {likelihood_name[safe.adult]}")
    print(f"medical: {likelihood_name[safe.medical]}")
    print(f"spoofed: {likelihood_name[safe.spoof]}")
    print(f"violence: {likelihood_name[safe.violence]}")
    print(f"racy: {likelihood_name[safe.racy]}")

    labels = {
        "adult": likelihood_name[safe.adult],
        "medical": likelihood_name[safe.medical],
        "spoofed": likelihood_name[safe.spoof],
        "violence": likelihood_name[safe.violence],
        "racy": likelihood_name[safe.racy],
    }

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
        
    return labels



In [ ]:
detect_safe_search_local("/Downloads/WomensHealthImages/resized/test_set/dreamstime_m_179640101.jpg")

Safe search:
adult: UNLIKELY
medical: UNLIKELY
spoofed: VERY_UNLIKELY
violence: UNLIKELY
racy: LIKELY


{'adult': 'UNLIKELY',
 'medical': 'UNLIKELY',
 'spoofed': 'VERY_UNLIKELY',
 'violence': 'UNLIKELY',
 'racy': 'LIKELY'}

In [ ]:
# label all iamges in the test set...
import glob

files = []
labels = []
for filename in glob.glob("/Downloads/WomensHealthImages/resized/test_set/*/*.jpg"):
    print(f"filename: {filename}")
    files.append(filename)
    labels.append(detect_safe_search_local(filename))

filename: /Users/duncan/Downloads/WomensHealthImages/resized/test_set/ReproductiveHealth/dreamstimemaximum_45400686.jpg
Safe search:
adult: POSSIBLE
medical: LIKELY
spoofed: VERY_UNLIKELY
violence: UNLIKELY
racy: VERY_LIKELY
filename: /Users/duncan/Downloads/WomensHealthImages/resized/test_set/WomensHealthGeneral/dreamstimemaximum_148999817.jpg
Safe search:
adult: VERY_UNLIKELY
medical: UNLIKELY
spoofed: VERY_UNLIKELY
violence: UNLIKELY
racy: LIKELY
filename: /Users/duncan/Downloads/WomensHealthImages/resized/test_set/WomensHealthGeneral/dreamstimemaximum_165473986.jpg
Safe search:
adult: UNLIKELY
medical: UNLIKELY
spoofed: VERY_UNLIKELY
violence: VERY_UNLIKELY
racy: VERY_LIKELY
filename: /Users/duncan/Downloads/WomensHealthImages/resized/test_set/WomensHealthGeneral/dreamstimemaximum_121070767.jpg
Safe search:
adult: POSSIBLE
medical: LIKELY
spoofed: UNLIKELY
violence: UNLIKELY
racy: VERY_LIKELY
filename: /Users/duncan/Downloads/WomensHealthImages/resized/test_set/WomensHealthGeneral/

In [ ]:
# write labels to file
import json
with open("./labeling_results/google_moderation_labels.tsv", "w") as f:
    for image, l in zip(files, labels):
        f.write(image[len("/Downloads/WomensHealthImages/resized/test_set/"):] + "\t" + json.dumps(l) + "\n")